In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import minmax_scale, OneHotEncoder, StandardScaler, robust_scale

from sklearn.kernel_approximation import Nystroem
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.utils import compute_sample_weight
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import roc_auc_score

#sys.path.append('/mnt/c/Users/conor/Git_Projects/PHD/')

from Preprocessing_Methods import *

from TSquared.hotelling_t2 import HotellingT2

from tqdm.notebook import tqdm

/home/conor/miniconda3/envs/PHD/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
total = pd.read_pickle('./December_Datat_32.pickle')
total = truncate(total, start=1000, end=1800)

ht2 = HotellingT2().fit(total)
#total = total.loc[ht2.predict(total)==1,:]

min_ = total.index.get_level_values('Patient_Number').value_counts().min()
sel = total#.groupby('Patient_Number').sample(min_)
sel = sel.reset_index(['ASMA']).dropna()#.sample(2000)
sel['IDX'] = np.arange(sel.shape[0])
sel = sel.set_index('IDX', append=True)

sel.columns = [str(col) for col in sel.columns]

In [4]:
def create_pipe():

    numeric_pipe = Pipeline([
    ("Normalise spectra", FunctionTransformer(minmax_scale, kw_args = {"axis": 1})),
    ("Feature Scale", StandardScaler()),
    ("PCA", PCA(5)),
    ])

    categorical_pipe = Pipeline([
        ("OneHot", OneHotEncoder(sparse=False, handle_unknown='ignore', dtype=int))
    ])

    ct = make_column_transformer(
        (numeric_pipe,     make_column_selector(dtype_include=np.number)),
        (categorical_pipe, make_column_selector(dtype_include=object))
    )

    #pipe = Pipeline([('Preprocess', ct), ('Classifier', LogisticRegression())])
    pipe = Pipeline([('Preprocess', ct), ('Classifier', CalibratedClassifierCV(LinearDiscriminantAnalysis(), cv=3, method="isotonic"))])

    return pipe

In [5]:
def create_pipe():

    numeric_pipe = Pipeline([
    ("Normalise spectra", FunctionTransformer(minmax_scale, kw_args = {"axis": 1})),
    ("Feature Scale", StandardScaler()),
    ("PCA", PCA(0.99))
    ])

    categorical_pipe = Pipeline([
        ("OneHot", OneHotEncoder(sparse=False, handle_unknown='ignore', dtype=int))
    ])

    ct = make_column_transformer(
        (numeric_pipe,     make_column_selector(dtype_include=np.number)),
        (categorical_pipe, make_column_selector(dtype_include=object))
    )

    #pipe = Pipeline([('Preprocess', ct), ('Classifier', LogisticRegression())])
    pipe = Pipeline([('Preprocess', ct), ('Classifier', LogisticRegression())])

    return pipe

In [6]:
bootstrap_pats = pd.read_pickle('./train_indices_20_December.pickle')

def bootstrap_it(n):

    for idx, row in bootstrap_pats.iloc[:n,:].iterrows():

        train_i = sel.query(f"Patient_Number in {list(row.iloc[0])}").reset_index()['IDX'].values
        test_i = sel.query(f"Patient_Number in {list(row.iloc[1])}").reset_index()['IDX'].values 

        yield train_i, test_i

In [7]:
for y_label in ['1yeardeath', '2year', 'Opt_Group']:

    patient_weights = compute_sample_weight('balanced', sel.reset_index()['Patient_Number'])
    label_weights = compute_sample_weight('balanced', sel.reset_index()[y_label])

    weights = patient_weights * label_weights

    for name, X_data in zip(['ASMA', 'ASMA_FTIR', 'FTIR'], [sel.iloc[:,[0]], sel, sel.iloc[:,1:]]):

        straps = dict()

        for i, (train_i, test_i) in tqdm(enumerate(bootstrap_it(100))):
            
            pipe = create_pipe()
            #pipe.fit(X_data.iloc[train_i,:], sel.iloc[train_i,:].index.get_level_values(f'{y_label}').astype(int), **{'Classifier__sample_weight': weights[train_i]})
            pipe.fit(X_data.iloc[train_i,:], sel.iloc[train_i,:].index.get_level_values(f'{y_label}').astype(int), **{'Classifier__sample_weight': weights[train_i]})


            pred_df = pd.DataFrame(pipe.predict_proba(X_data.iloc[test_i])[:,1], columns=['Preds'], index=sel.iloc[test_i].index)
            pred_df['Y_true'] = sel.iloc[test_i,:].index.get_level_values(f'{y_label}').astype(int)
            pred_df['Weights'] = weights[test_i]

            straps[i] = pred_df.astype(np.float16)

        save_path = f'./Results/A/{y_label}/{name}'

        if not os.path.exists(save_path):

            os.makedirs(save_path)
            

        pd.concat(straps).to_pickle(f'./Results/A/{y_label}/{name}/results.pickle')

/home/conor/miniconda3/envs/PHD/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/conor/miniconda3/envs/PHD/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

/home/conor/miniconda3/envs/PHD/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/conor/miniconda3/envs/PHD/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

/home/conor/miniconda3/envs/PHD/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/conor/miniconda3/envs/PHD/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.